In [22]:
import requests
import csv
import pandas as pd
import json
import dateutil
import time
import json

rest_api_domain = "https://api-fxpractice.oanda.com"

def get_token():
    f = open('../.practice_token', 'r')
    return str(f.read())

access_token = "";

#access_token = get_token()
print(access_token)
account_id = "101-004-7361006-006"
granularity = "D"
instrument = "GBP_USD"
number_of_candles = "180"
candles_endpoint = f'/v3/instruments/{instrument}/candles?count={number_of_candles}&granularity={granularity}'
authorization_header = f'Bearer {access_token}'
trades_endpoint = f"/v3/accounts/{account_id}/trades"

In [26]:
# Docs: http://developer.oanda.com/rest-live-v20/trade-ep/
# For analysis of closed trades: '?instrument=AUD_USD&state=CLOSED&count=500'
def get_trades():
    global rest_api_domain, trades_endpoint
    url = rest_api_domain + trades_endpoint
    response = requests.get(url, headers={'Authorization': authorization_header})
    response_json = response.json()
    trades = response_json['trades']
    return trades

In [43]:
def trades_by_instrument(instrument, state='OPEN'): #open or closed
    global rest_api_domain, trades_endpoint
    # Should not do a call per instrument
    url = rest_api_domain + trades_endpoint + f"?instrument={instrument}&state={state}&count=500"
    response = requests.get(url, headers={'Authorization': authorization_header})
    response_json = response.json()
    trades = response_json['trades']
    return trades

In [3]:
def datetime_to_utc_timestamp(date_time):
    return int(time.mktime(date_time.timetuple()))

In [56]:
# receives the time as 'openTime', 'closedTime', 'time' from trade/candle data from Oanda and gets back the unix utc
def time_in_response_to_utc(time_in_response):
    parsed_date_time = dateutil.parser.parse(time_in_response)
    date_time_utc = datetime_to_utc_timestamp(parsed_date_time)
    return date_time_utc

In [70]:
def financing_per_instrument(instrument, state='OPEN'):
    trades = trades_by_instrument(instrument=instrument, state=state)
    financing_dict = { 'financing': sum(float(trade['financing']) for trade in trades), 'instrument': instrument, 'trades': len(trades)}
    return financing_dict

In [75]:
instruments = ["NZD_USD", "AUD_USD", "USD_MXN", "GBP_JPY", "GBP_USD", "AUD_JPY", "USD_JPY", "EUR_JPY", "EUR_USD"]
state = "OPEN"

financings = [ financing_per_instrument(instr, state) for instr in instruments]




In [76]:
financings

[{'financing': -21.549999999999997, 'instrument': 'NZD_USD', 'trades': 13},
 {'financing': -42.6399, 'instrument': 'AUD_USD', 'trades': 12},
 {'financing': -11.235700000000001, 'instrument': 'USD_MXN', 'trades': 2},
 {'financing': 2.4109, 'instrument': 'GBP_JPY', 'trades': 8},
 {'financing': -14.4807, 'instrument': 'GBP_USD', 'trades': 3},
 {'financing': 8.8803, 'instrument': 'AUD_JPY', 'trades': 8},
 {'financing': -7.6377999999999995, 'instrument': 'USD_JPY', 'trades': 2},
 {'financing': -5.371, 'instrument': 'EUR_JPY', 'trades': 6},
 {'financing': -28.1661, 'instrument': 'EUR_USD', 'trades': 4}]

In [4]:
def extract_candlesticks(response_json):
    candlesticks = []
    for candle in response_json['candles']:
        p_open = float(candle['mid']['o'])
        p_close = float(candle['mid']['c'])
        p_low = float(candle['mid']['l'])
        p_high = float(candle['mid']['h'])
        the_time = candle['time']
        parsed_date_time = dateutil.parser.parse(the_time)
        date_time_utc = datetime_to_utc_timestamp(parsed_date_time)
        candlestick = { 'open': p_open, 'close': p_close, 'low': p_low, 'high': p_high, 'time': date_time_utc}
        candlesticks.append(candlestick)
    return {'instrument': response_json['instrument'], 'granularity': response_json['granularity'], 'candles': candlesticks}
    

In [5]:
def load_candles_as_dict(json_data):
    return json.loads(json_data)['candles']

def read_json_file(filepath):
    with open(filepath, 'r') as thefile:
        data = thefile.read()
        return load_candles_as_dict(data)

In [6]:
the_time = r['candles'][0]['time']
print(the_time)
#print(time.mktime(int(the_time)))
print(time.strftime("%x", time.gmtime(int(the_time))))

NameError: name 'r' is not defined

In [10]:
def format_date(date_as_timestamp):
    return time.strftime("%x", time.gmtime(int(date_as_timestamp)))

In [11]:
def write_to_json(json_data_as_dict):
    with open('/Users/david/Desktop/json_data.json', 'w') as fp:
        json.dump(json_data_as_dict, fp)

In [12]:
def write_to_csv(candles):
     with open('/Users/david/Desktop/csv_data1.csv', 'w') as csvfile:
        spamwriter = csv.writer(csvfile,delimiter=',')
        for candle in candles:
            #print(f'Candle:' + str(candle['close']))
            date = format_date(candle['time'])
            data_to_write = [date,candle['close']]
            print(data_to_write)
            spamwriter.writerow(data_to_write)
        

In [37]:
candles = read_json_file('/Users/david/Desktop/json_data.json')

In [13]:
def read_as_pandas_df(trades_json):
    df = pd.read_json(json.dumps(trades_json))
    return df

In [14]:
def parse_trade_from_response(trade, for_open_trades=True):
    trade_list = []
    instrument = trade['instrument']
    open_time = trade['openTime']
    units = int(trade['initialUnits'])
    direction = "Buy" if units > 0 else "Sell"
    financing = trade['financing']
    realized_pl = trade['realizedPL']
    initial_margin_req = trade['initialMarginRequired']
    open_price = trade['price']
    take_profit = 'Not Set'
    stop_loss = 'Not Set'
    if 'takeProfitOrder' in trade:
        take_profit = trade['takeProfitOrder']['price']
    if 'stopLossOrder' in trade:
        stop_loss = trade['stopLossOrder']['price']
    if for_open_trades:
        margin_used = trade['marginUsed']
        unrealized_pl = trade['unrealizedPL']
        trade_list = [instrument, open_time, float(open_price), direction, units, unrealized_pl, take_profit, stop_loss, float(financing)]
    else:
        close_time = trade['closeTime']
        trade_list = [instrument, open_time, close_time, float(open_price), direction, units, take_profit, stop_loss, float(financing)]
    return trade_list

In [15]:
def get_csv_header():
    return [str("Instrument"), str('Open Time'), "Open Price", "Direction","Units", "P/L", "TP", "SL", "Financing"]

In [16]:
def write_csv(filename,rows_as_lists):
    with open(filename, 'w') as csvfile:
        spamwriter = csv.writer(csvfile,delimiter=',')
        for row in rows_as_lists:
            spamwriter.writerow(row)

In [20]:
trades = get_trades()
open_trades = True
all_trades = [ parse_trade_from_response(t,open_trades) for t in trades]

In [101]:
all_trades = [get_csv_header()] + all_trades

In [102]:
write_csv("/Users/david/Desktop/trades.csv",all_trades)

In [27]:
# ======== Only for current open trades ======
# https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
trades = get_trades()
df = read_as_pandas_df(trades)
#df['Direction'] = "Buy" if int(df['currentUnits']) > 0 else "Sell"
df['Direction'] = df.apply(lambda row: "Buy" if int(row['currentUnits']) > 0 else "Sell",axis=1)
df['currentUnits'] = df.apply(lambda row: -1*int(row['currentUnits']) if int(row['currentUnits']) < 0 else int(row['currentUnits']),axis=1)
#df['openTime'] = df['openTime'].apply(dateutil.parser.parse, dayfirst=True)
buys = df['currentUnits'][df['Direction'] == 'Buy'].sum()
sells = df['currentUnits'][df['Direction'] == 'Sell'].sum()
direction_counts = df['Direction'].value_counts()
print("Directions:")
print(direction_counts)
print("=================")
print(f"Units bought: {buys}")
print(f"Units sold: {sells}")


KeyError: 'trades'

In [24]:
df_grouped = df.groupby(['instrument','Direction'],as_index=False)
df2 = df_grouped.agg({'currentUnits':'sum', 'financing': 'sum', 'realizedPL': 'sum'})
# df2 = df_grouped.agg({'initialUnits':'sum', 'financing': 'sum', 'realizedPL': 'sum'})
# df2['perc'] = df2.apply(lambda row: 100*row['currentUnits']/buys if row['Direction'] == 'Buy' else 100*row['currentUnits']/sells,axis=1)
df2


NameError: name 'df' is not defined